# When Real Yields Rise but Gold Doesn’t Blink
### A decoupling worth your attention—and a simple regime map to trade it.


In [1]:
from fredapi import Fred
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

# API key
fred = Fred(api_key="7f8e44038ee69c4f78cf71873e85db16")

# Colors
COLORS = {
    "blue": "#0067db",
    "orange": "#ff8c42",
    "purple": "#6a0dad",
    "gray": "#7a7a7a",
}

# Create charts directory
charts_dir = Path("charts")
charts_dir.mkdir(exist_ok=True)
print(f"Charts will be saved to: {charts_dir.absolute()}")

Charts will be saved to: /Users/bob/charts


In [ ]:
# %% [single cell] Lighthouse Macro Notebook — FRED + Alpha Vantage end-to-end
# - Pulls macro series from FRED (real yields, USD, CPI, etc.)
# - Uses Alpha Vantage (AV) as well for assets FRED doesn't cover or as proxies (e.g., GLD, UUP, XAUUSD)
# - Builds charts in Lighthouse Macro palette
# - Computes rolling correlations and regime betas (by real yield level/trend)
# - Writes a full Markdown blog draft that references the saved charts
#
# Prereqs (run once in your environment if needed):
#   pip install fredapi pandas numpy matplotlib requests statsmodels

from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import time
from datetime import date

# Try to import statsmodels, fallback to scipy if not available
try:
    from statsmodels.api import OLS, add_constant
    HAS_STATSMODELS = True
except ImportError:
    print("Warning: statsmodels not available. Using simple correlation instead of OLS for beta calculations.")
    HAS_STATSMODELS = False

# =========================
# CONFIG & KEYS
# =========================
FRED_API_KEY = "7f8e44038ee69c4f78cf71873e85db16"                   # provided
ALPHAVANTAGE_API_KEY = "IOTZFZG01XK55BHI"                           # provided
START_DATE = "2015-01-01"                                           # adjust the analysis window
TITLE = "When Real Yields Rise but Gold Doesn't Blink"
SUBTITLE = "A decoupling worth your attention—and a simple regime map to trade it."

# Branding palette
COLORS = {
    "blue":   "#0067db",  # primary
    "orange": "#ff8c42",  # medium-dark but bright orange
    "purple": "#6a0dad",  # deep purple
    "gray":   "#7a7a7a",  # medium-dark gray
}

plt.rcParams.update({
    "axes.edgecolor": "#333333",
    "axes.titleweight": "bold",
    "axes.labelcolor": "#333333",
    "font.size": 12,
    "figure.dpi": 150,
    "savefig.dpi": 180,
})

# Output paths
OUT = Path("lighthouse_outputs")
CH = OUT / "charts"
TB = OUT / "tables"
OUT.mkdir(exist_ok=True, parents=True)
CH.mkdir(exist_ok=True, parents=True)
TB.mkdir(exist_ok=True, parents=True)

# =========================
# HELPERS
# =========================
def monthly(series: pd.Series) -> pd.Series:
    """Coerce to month-end frequency with last available obs."""
    s = pd.Series(series).dropna()
    s.index = pd.to_datetime(s.index)
    return s.resample("ME").last().ffill()

def clip(s: pd.Series, start=START_DATE) -> pd.Series:
    return s[s.index >= pd.to_datetime(start)]

def index_100(s: pd.Series) -> pd.Series:
    s = s.dropna()
    return s / s.iloc[0] * 100.0

def pct_change_12m(s: pd.Series) -> pd.Series:
    return s.pct_change(12) * 100.0

def monthly_returns(s: pd.Series) -> pd.Series:
    return s.pct_change(1).rename(s.name + "_ret")

def delta(s: pd.Series) -> pd.Series:
    """Simple first difference (e.g., monthly change in yields)."""
    return s.diff(1).rename(s.name + "_chg")

def roll_corr(a: pd.Series, b: pd.Series, window=12) -> pd.Series:
    return a.rolling(window).corr(b)

def ols_beta(y: pd.Series, x: pd.Series):
    """OLS beta of y on x with t-stat and n-obs. Fallback to correlation if statsmodels unavailable."""
    df = pd.concat([y, x], axis=1).dropna()
    if len(df) < 12:
        return np.nan, np.nan, np.nan
    
    if HAS_STATSMODELS:
        X = add_constant(df.iloc[:,1].values)
        model = OLS(df.iloc[:,0].values, X).fit()
        return float(model.params[1]), float(model.tvalues[1]), int(model.nobs)
    else:
        # Simple correlation-based beta approximation
        corr = df.iloc[:,0].corr(df.iloc[:,1])
        std_ratio = df.iloc[:,0].std() / df.iloc[:,1].std()
        beta = corr * std_ratio
        # Rough t-stat approximation
        n = len(df)
        t_stat = corr * np.sqrt((n-2)/(1-corr**2)) if abs(corr) < 0.99 else np.nan
        return float(beta), float(t_stat), int(n)

def to_monthly_from_daily(df: pd.DataFrame, value_col: str) -> pd.Series:
    """Convert daily AV time series to monthly last."""
    s = df[value_col].copy()
    s.index = pd.to_datetime(s.index)
    s = s.sort_index().resample("ME").last().ffill()
    return s

def av_get(symbol: str, function: str, **params) -> dict:
    """Generic Alpha Vantage GET with basic retry/backoff."""
    base = "https://www.alphavantage.co/query"
    payload = {"function": function, "apikey": ALPHAVANTAGE_API_KEY}
    payload.update(params)
    for i in range(3):
        r = requests.get(base, params=payload, timeout=30)
        if r.status_code == 200:
            data = r.json()
            # Handle throttling / note
            if any(k in data for k in ["Error Message", "Information", "Note"]):
                time.sleep(12 * (i + 1))
                continue
            return data
        time.sleep(3 * (i + 1))
    return {}

def av_fx_daily(from_symbol: str, to_symbol: str) -> pd.DataFrame:
    """FX daily series (e.g., XAUUSD if supported)."""
    data = av_get(function="FX_DAILY", from_symbol=from_symbol, to_symbol=to_symbol, outputsize="full")
    key = "Time Series FX (Daily)"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close"
    }).apply(pd.to_numeric, errors="coerce")
    df.index.name = "date"
    return df

def av_equity_monthly(symbol: str) -> pd.DataFrame:
    """Monthly OHLC for ETFs/equities (e.g., GLD, UUP)"""
    data = av_get(function="TIME_SERIES_MONTHLY_ADJUSTED", symbol=symbol)
    key = "Monthly Adjusted Time Series"
    if key not in data:
        return pd.DataFrame()
    df = pd.DataFrame(data[key]).T.rename(columns={
        "1. open":"open", "2. high":"high", "3. low":"low", "4. close":"close",
        "5. adjusted close":"adj_close", "6. volume":"volume", "7. dividend amount":"dividend"
    }).apply(pd.to_numeric, errors="coerce")
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

def safe_plot(series_list, labels, colors, title, ylab, fname, hline0=False):
    plt.figure(figsize=(9,5))
    for s, lab, col in zip(series_list, labels, colors):
        s.dropna().plot(label=lab, color=col)
    if hline0:
        plt.axhline(0, linestyle="--", color=COLORS["gray"])
    plt.title(title)
    plt.ylabel(ylab); plt.xlabel("")
    plt.legend()
    plt.tight_layout()
    plt.savefig(CH / fname)
    plt.close()

print("Setup complete! Ready to fetch data...")
if not HAS_STATSMODELS:
    print("Note: Using correlation-based beta approximation instead of full OLS regression.")

SyntaxError: invalid syntax (2175418124.py, line 9)

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [ ]:
# =========================
# FETCH DATA — FRED first
# =========================
fred = Fred(api_key=FRED_API_KEY)

# Deep-dive backbone - try multiple gold series
print("Fetching FRED data...")

# Get TIPS real yield
dfii10 = clip(monthly(fred.get_series("DFII10")))                 # 10Y TIPS real yield (%)
print(f"✓ Got TIPS real yield: {len(dfii10)} observations")

# Try to get USD broad index
try:
    usd_broad = clip(monthly(fred.get_series("DTWEXBGS")))            # Trade-Weighted USD: Broad
    print(f"✓ Got USD broad index: {len(usd_broad)} observations")
except:
    print("⚠ Could not fetch USD broad index, will use proxy")
    usd_broad = pd.Series(dtype=float)

# Try multiple gold series
gold_fred = pd.Series(dtype=float)
gold_series_to_try = [
    ("GOLDAMGBD228NLBM", "Gold AM London Fix"),
    ("GOLDPMGBD228NLBM", "Gold PM London Fix"), 
    ("GOLDS", "Gold Handy & Harman Base Price"),
]

for series_id, description in gold_series_to_try:
    try:
        gold_fred = clip(monthly(fred.get_series(series_id)))
        print(f"✓ Got {description}: {len(gold_fred)} observations")
        break
    except:
        print(f"⚠ Could not fetch {description} ({series_id})")
        continue

# Dashboard indicators
try:
    payems = clip(monthly(fred.get_series("PAYEMS")))                 # Nonfarm Payrolls (thousands)
    unrate = clip(monthly(fred.get_series("UNRATE")))                 # Unemployment rate (%)
    cpi    = clip(monthly(fred.get_series("CPIAUCSL")))               # CPI Index
    core   = clip(monthly(fred.get_series("CPILFESL")))               # Core CPI Index
    lei    = clip(monthly(fred.get_series("USSLIND")))                # Leading Index (Conference Board)
    umich  = clip(monthly(fred.get_series("UMCSENT")))                # U. Michigan Sentiment
    print("✓ Got dashboard indicators")
except Exception as e:
    print(f"⚠ Some dashboard indicators failed: {e}")

print("\nFRED data fetch complete!")
print(f"Analysis period: {START_DATE} onwards")
print(f"Real yield observations: {len(dfii10)}")
print(f"Gold observations: {len(gold_fred)}")